save the previous code to a model

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X=np.array(X/255.0) #the max value of pixel is 255
y=np.array(y)

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('Rama2-Catdog-CNN.model')

3-conv-64-nodes-0-dense-1611560850
Epoch 1/10
546/546 [==============================] - 184s 335ms/step - loss: 0.6674 - accuracy: 0.5790 - val_loss: 0.5601 - val_accuracy: 0.7155
Epoch 2/10
546/546 [==============================] - 205s 376ms/step - loss: 0.5363 - accuracy: 0.7293 - val_loss: 0.5051 - val_accuracy: 0.7586
Epoch 3/10
546/546 [==============================] - 198s 364ms/step - loss: 0.4593 - accuracy: 0.7855 - val_loss: 0.4402 - val_accuracy: 0.7924
Epoch 4/10
546/546 [==============================] - 202s 370ms/step - loss: 0.3999 - accuracy: 0.8158 - val_loss: 0.4274 - val_accuracy: 0.7997
Epoch 5/10
546/546 [==============================] - 200s 367ms/step - loss: 0.3676 - accuracy: 0.8383 - val_loss: 0.3796 - val_accuracy: 0.8314
Epoch 6/10
546/546 [==============================] - 194s 356ms/step - loss: 0.3326 - accuracy: 0.8563 - val_loss: 0.3681 - val_accuracy: 0.8371
Epoch 7/10
546/546 [==============================] - 204s 374ms/step - loss: 0.2980 - ac

define a function for testing the model with new picture, predict the picture

In [8]:
import cv2
import tensorflow as tf

CATEGORIES = ['Dog','Cat']

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model('Rama2-Catdog-CNN.model')

prediction = model.predict([prepare('D:/Rama/AI Coaching/catdog/PetImages/dog.jpg')])
print(prediction)

[[0.]]


Make it to categories, so it prints cat/dog

In [9]:
prediction = model.predict([prepare('D:/Rama/AI Coaching/catdog/PetImages/dog.jpg')])
print(CATEGORIES[int(prediction[0][0])])

Dog


In [10]:
prediction = model.predict([prepare('D:/Rama/AI Coaching/catdog/PetImages/cat.jpg')])
print(CATEGORIES[int(prediction[0][0])])

Cat
